In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
!pip install git+https://github.com/ultralytics/ultralytics.git@main
!pip install supervision
!pip install --upgrade pytube
!pip install -q ultralytics PyYAML
!pip install yt-dlp

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-8hz_eyqa
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-8hz_eyqa
  Resolved https://github.com/ultralytics/ultralytics.git to commit 3d4a86d2ee35e52b99a65f35126f758078fcccac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.66-py3-none-any.whl size=911405 sha256=b41e6f8367d2c54b1601b0080a4817e582994582be0b2095c4c8d8fdda137224
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zhildxy/wheels/ce/28/0b/747d53a7fba3579e3367c6a4b40abf971babc68f7a46b16a28
Successfully built ultralytics
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# download youtube videos

import yt_dlp

# Options to ensure download in mp4 format
ydl_opts = {
    'format': 'mp4',  # Ensure the download format is mp4
    'outtmpl': '/content/drive/MyDrive/Colab Notebooks/videos/%(title)s.%(ext)s',  # Save with .mp4 extension
}

# URL of the YouTube video
video_url = "https://www.youtube.com/watch?v=eZDhGCdwP8U"

# Download the video
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

print(f"Video has been successfully downloaded to /content/drive/MyDrive/Colab Notebooks/videos")

[youtube] Extracting URL: https://www.youtube.com/watch?v=eZDhGCdwP8U
[youtube] eZDhGCdwP8U: Downloading webpage
[youtube] eZDhGCdwP8U: Downloading ios player API JSON
[youtube] eZDhGCdwP8U: Downloading web creator player API JSON
[youtube] eZDhGCdwP8U: Downloading m3u8 information
[info] eZDhGCdwP8U: Downloading 1 format(s): 18
[download] Destination: /content/drive/MyDrive/Colab Notebooks/videos/UK Dash Cam #9 - Blue Lights [Police, Ambulances & Fire] Responding 999.mp4
[download] 100% of   26.03MiB in 00:00:07 at 3.40MiB/s   
Video has been successfully downloaded to /content/drive/MyDrive/Colab Notebooks/videos


In [ ]:
# Define the path to the video and image directories
VIDEO_DIR_PATH = "/content/drive/My Drive/Colab Notebooks/videos"
IMAGE_DIR_PATH = "/content/drive/My Drive/Colab Notebooks/images"
FRAME_STRIDE = 10  # Adjust this to save every Nth frame

In [ ]:
# Convert Videos to Images/frames

import cv2
import os
from tqdm import tqdm  # Import tqdm for the progress bar
import shutil

def clear_directory(path):
    # List all files and directories in the given directory
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            # Check if it is a file or a link, then delete it
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # If it is a directory, remove the entire directory tree
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Ensure the target directory for images exists and is clear
os.makedirs(IMAGE_DIR_PATH, exist_ok=True)
clear_directory(IMAGE_DIR_PATH)  # Optional: clear the directory to ensure it contains only current video frames

# Input and check video file
video_file = input("Enter the video file name (including extension, e.g., video.mp4): ")
video_path = os.path.join(VIDEO_DIR_PATH, video_file)
if not os.path.exists(video_path):
    print(f"Video file {video_path} not found.")
else:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
    else:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_count = 0
        with tqdm(total=total_frames // FRAME_STRIDE, desc="Extracting Frames") as pbar:
            while True:
                success, frame = cap.read()
                if not success:
                    break
                if frame_count % FRAME_STRIDE == 0:
                    img_name = f"frame{frame_count}.jpg"  # Updated to save as 'frame{frame_number}.jpg'
                    cv2.imwrite(os.path.join(IMAGE_DIR_PATH, img_name), frame)
                    pbar.update(1)
                frame_count += 1
        cap.release()
        print("Conversion complete.")

Enter the video file name (including extension, e.g., video.mp4): UK Dash Cam #9 - Blue Lights [Police, Ambulances & Fire] Responding 999.mp4


Extracting Frames: 1488it [00:26, 55.21it/s]

Conversion complete.


In [ ]:
import supervision as sv

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["jpg"])

print('image count:', len(image_paths))

image count: 1488


In [ ]:
# plot some of the extracted frames

import cv2
import supervision as sv

SAMPLE_SIZE = 16
SAMPLE_GRID_SIZE = (4, 4)
SAMPLE_PLOT_SIZE = (16, 16)

titles = [
    image_path.stem
    for image_path
    in image_paths[:SAMPLE_SIZE]]
images = [
    cv2.imread(str(image_path))
    for image_path
    in image_paths[:SAMPLE_SIZE]]

sv.plot_images_grid(images=images, titles=titles, grid_size=SAMPLE_GRID_SIZE, size=SAMPLE_PLOT_SIZE)

In [ ]:
# annotate these images

from ultralytics import YOLO
import os
import shutil
from pathlib import Path

def clear_directory(path):
    # List all files and directories in the given directory
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            # Check if it is a file or a link, then delete it
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # If it is a directory, remove the entire directory tree
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Load a pretrained YOLOv8n model
model = YOLO('yolov8l.pt')

# Specify the classes to detect (indices for 'car', 'motorcycle', 'bus', 'truck')
classes_to_detect = [2, 3, 5, 7]

# Define paths
source = '/content/drive/My Drive/Colab Notebooks/images'
annotated_images_path = '/content/drive/My Drive/Colab Notebooks/annotated_images_test_large'
labels_path = '/content/drive/My Drive/Colab Notebooks/labels_test_large'

# Clear directories
clear_directory(annotated_images_path)
clear_directory(labels_path)

# Ensure output directories exist
os.makedirs(annotated_images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# Get the list of image filenames and sort them numerically
image_filenames = sorted(Path(source).glob('*.jpg'), key=lambda x: int(x.stem.split('frame')[-1]))

# Run inference on images
for index, image_path in enumerate(image_filenames):

    # Extract base name from the image file (e.g., 'Dashcam SWE #3_frame0.jpg' => 'Dashcam SWE #3')
    base_name = image_path.stem.rsplit('_frame', 1)[0]

    # Inference arguments
    results = model.predict(str(image_path), stream=True, classes=classes_to_detect, conf=0.25, imgsz=640, iou=0.5, agnostic_nms=True)

    # Process results
    for result in results:
        # Extract the bounding boxes and their respective class names from the inference results.
        boxes = result.boxes.xywhn  # BB coordinates (normalized by original image size)
        class_labels = result.boxes.cls  # Return the class values of the boxes.

        # Extract frame number from the file name
        frame_number = image_path.stem.split('frame')[-1]

        # Construct the annotation file name using the base name
        annotation_filename = os.path.join(labels_path, f'{base_name}_frame{frame_number}.txt')
        with open(annotation_filename, 'w') as f:
            for box, cls_id in zip(boxes, class_labels):
                f.write(f'{int(cls_id)} {box[0]} {box[1]} {box[2]} {box[3]}\n')

        # Save annotated image
        annotated_image_filename = os.path.join(annotated_images_path, f'{base_name}_frame{frame_number}.jpg')
        result.save(filename=annotated_image_filename, labels=False, line_width=1)  # Save image with bounding boxes but no labels

    # Break the loop if the last frame is reached
    if index == len(image_filenames) - 1:
        break